In [11]:
import os

from pandas.io.parquet import to_parquet

In [12]:
%pwd


'D:\\HourlyConsumptiomXGboost'

In [13]:
os.chdir(r"D:\HourlyConsumptiomXGboost")

In [14]:
%pwd

'D:\\HourlyConsumptiomXGboost'

In [15]:
# entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    # source_URL:str
    local_data_file: Path

In [16]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            # source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )

        return data_ingestion_config

In [18]:
import os
import urllib.request as request
import pandas as pd
from src.mlProject import logger
from src.mlProject.utils.common import get_size,get_mongoData


In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
           
    def initiateDataIngestion(self):
        try:
            data = get_mongoData()
            logger.info("Reading Completed from MongoDB")
            logger.info("Writing data in Parquet file")
            data.to_parquet(self.config.local_data_file, index=False, engine='fastparquet')
            read_df = pd.read_parquet(self.config.local_data_file)
            print(read_df)
            
        except Exception as e:
            print(f"Error: {e}")
  

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.initiateDataIngestion()
    # data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-11 17:54:25,525: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-11 17:54:25,527: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-11 17:54:25,529: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-11 17:54:25,531: INFO: common: created directory at: artifacts]
[2023-10-11 17:54:25,532: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-11 17:54:25,534: INFO: common: calling DB configuration]
[2023-10-11 17:54:25,535: INFO: common: Reading data from Mongo DB]


--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\Python311\Lib\logging\__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python311\Lib\logging\__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python311\Lib\logging\__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\Python311\Lib\logging\__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "D:\HourlyConsumptiomXGboost\venv\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "D:\HourlyConsumptiomXGboost\venv\Lib\si

===============DataType Conversion==================
[2023-10-11 17:54:38,344: INFO: 21609390: Reading Completed from MongoDB]
[2023-10-11 17:54:38,345: INFO: 21609390: Writing data in Parquet file]
          location_id       creation_time  grid_reading_kwh
0        500000010032 2021-05-01 00:01:57             460.0
1        500000010032 2021-05-01 00:04:01             460.0
2        500000010032 2021-05-01 00:06:06             460.0
3        500000010032 2021-05-01 00:08:12             460.0
4        500000010032 2021-05-01 00:10:17             461.0
...               ...                 ...               ...
1041507  500000010832 2023-08-17 23:15:33           12398.0
1041508  500000010832 2023-08-17 23:25:16           12398.0
1041509  500000010832 2023-08-17 23:34:58           12399.0
1041510  500000010832 2023-08-17 23:44:44           12399.0
1041511  500000010832 2023-08-17 23:54:26           12399.0

[1041512 rows x 3 columns]
